<a href="https://colab.research.google.com/github/askolesov0608/SJK006-Project/blob/main/SJK006_RF_Best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score

import logging
from tqdm.auto import tqdm
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


# Загрузка данных Loading data
file_path = 'TestDataCV_DigitalV2.xlsx'
data = pd.read_excel(file_path)

# Создание новой целевой переменной для должности Create a new target variable for position
positions = ['Human Resources', 'Research & Development', 'Sales',
             'Life Sciences', 'Marketing', 'Medical', 'Other']
data['Position'] = (data[positions] == 1).dot(range(1, len(positions) + 1))

# Выбор признаков для модели Selecting features for the model
features_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
                    'HourlyRate', 'JobInvolvement', 'JobLevel', 'GenderM', 'GenderF',
                    'Single', 'Married', 'Divorced']
features = data[features_columns]
target = data['Position']

# Бутстрэппинг каждого класса до 1000 записей Bootstrapping each class up to 1000 records
bootstrapped_data = pd.DataFrame(columns=features_columns + ['Position'])
for position in target.unique():
    position_data = data[data['Position'] == position]
    bootstrapped_sample = resample(position_data, replace=True, n_samples=1000, random_state=42)
    bootstrapped_data = pd.concat([bootstrapped_data, bootstrapped_sample])

X = bootstrapped_data[features_columns]
y = bootstrapped_data['Position'].astype(int)

# Масштабирование признаков Feature scaling
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Параметры для Randomized Search Options for Randomized Search
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50, None]
}

# Создание модели Random Forest для Randomized Search Creating a Random Forest Model for Randomized Search
rf_model = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(rf_model, param_distributions, n_iter=10, cv=5, random_state=42, verbose=3)

# Обучение с Randomized Search Training with Randomized Search
logger.info("Начало обучения start date RandomizedSearchCV")
random_search.fit(X_train, y_train)
logger.info("Обучение завершено Training completed")

# Лучшие параметры и модель Best parameters and model
best_rf_model = random_search.best_estimator_
print("Лучшие параметры модели Best model parameters:", random_search.best_params_)

# Сохранение лучшей модели Saving the best model
joblib.dump(best_rf_model, 'best_random_forest_model.joblib')
logger.info("Лучшая модель сохранена Best model saved")

# Prediction and evaluation of the best model
y_pred = best_rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Точность лучшей модели Accuracy of the best model: {accuracy}")
print("Отчет о классификации Classification report:")
print(report)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ..max_depth=None, n_estimators=300;, score=0.954 total time=   1.9s
[CV 2/5] END ..max_depth=None, n_estimators=300;, score=0.954 total time=   3.0s
[CV 3/5] END ..max_depth=None, n_estimators=300;, score=0.954 total time=   1.8s
[CV 4/5] END ..max_depth=None, n_estimators=300;, score=0.942 total time=   1.8s
[CV 5/5] END ..max_depth=None, n_estimators=300;, score=0.961 total time=   1.8s
[CV 1/5] END ....max_depth=40, n_estimators=100;, score=0.957 total time=   0.6s
[CV 2/5] END ....max_depth=40, n_estimators=100;, score=0.956 total time=   0.6s
[CV 3/5] END ....max_depth=40, n_estimators=100;, score=0.950 total time=   0.6s
[CV 4/5] END ....max_depth=40, n_estimators=100;, score=0.941 total time=   0.6s
[CV 5/5] END ....max_depth=40, n_estimators=100;, score=0.964 total time=   0.6s
[CV 1/5] END ....max_depth=50, n_estimators=400;, score=0.956 total time=   2.7s
[CV 2/5] END ....max_depth=50, n_estimators=400;